In [1]:
## Standard Imports ##
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
sns.set_context('talk')

import numpy as np
import scipy as sp

import time as timeit
import datetime
import sys,os

## Kalman Filter Imports ##
from filterpy.kalman import UnscentedKalmanFilter
from filterpy.kalman import MerweScaledSigmaPoints
from filterpy.kalman import unscented_transform

## Unscented Kalman Filter
In this file, I develop an unscented Kalman filter (UKF) solution to the identification of a single-degree-of-freedom oscillator (SDOF), in which the nonlinear switch state is turned OFF, essentially developing a linear, shear-frame SDOF system. The nonlinear stiffness contribution is turned off for this part of the example, as part of the switch state for this system. 

The UKF incorporates a small set of hyperparameters used to adjust the deterministic selection of the small set of sigma points used in the filtering approximation. For this example, these parameters are selected according to the "standard" values suggested by Wan and Van Der Merwe<sup>1</sup>, as listed below.
    
1. $\alpha = 0.001$, where $\alpha$ determines the spread of the sigma points about the mean of the latent state approximation. 
2. $\kappa = 0$, where $\kappa$ is a secondary scaling parameter on the sigma points. 
3. $\beta = 2$, where $\beta$ incorporates knowledge of the distribution on the latent states and $\beta = 2$ is the optimal representation for Gaussian distributions.  

Additional hyperparameters in an experimental scenario include the process noise covariance $Q$ and the process noise covariance $R$. However, these terms are assumed known for this problem. It should also be noted that where the EKF uses the a smoothed approximation of the dynamical model for the system, the UKF can operate on the non-smooth model directly. 

This example runs 50 inference trials using varied prior information on the parameters, simulating different assertions an experimentalist might make in a practical identification scenario. Outputs from this model include:
1. The mean and standard deviation of the state and log(parameters) over the inference period. 
2. The mean of the state and parameters over the inference period. 
3. The mode of the state and parameters over the inference period. 
4. The computational model response built from the inferred parameters with respect to the input signal used for inference.
5. The runtime for each inference trial. 

The UKF implementation expressed herein is drawn from the python library FilterPy<sup>2</sup>. Some small modifications were made to adapt the library to this problem, and are shown in detail in the code below. 

__Developed by__: Alana Lund (Purdue University) \
__Last Updated__: 22 Sept. 2021 \
__License__: AGPL-3.0

### References
<sup>1</sup> E. Wan and R. Van Der Merwe. The unscented Kalman filter for nonlinear estimation. _Proceedings of IEEE 2000 Adaptive Systems for Signal Processing, Communications, and Control Symposium._ (2000). pp 153-158. 

<sup>2</sup> R. Labbe, FilterPy v1.4.5, (2021). [https://github.com/rlabbe/filterpy](https://github.com/rlabbe/filterpy).

## Load Experimental Data

In [2]:
# File Names
inFile = '../04-Data/Linear/inferenceInput'
outFile = '../04-Data/Linear/outputUKF'

infData = np.load(inFile + '.npz')

dt = infData['dt']                 # time step [sec]
time = infData['time']             # time history [sec]
inpAcc = infData['inpAcc']         # observations on input acceleration [m/sec^2]
states = infData['statesPNoise']   # states (for post-inference validation) [m,m/sec]
respAcc = infData['accPMNoise']    # observations on response acceleration [m/sec^2]
Q = infData['Qfactor']             # process noise contributions, independent std. dev. per state [m,m/sec]
R = infData['Rfactor']             # measurement noise contribution [m/sec^2]
m = infData['m']                   # mass [kg]
ics = infData['ics']               # true initial conditions of the system [m, m/sec]
par = infData['par']               # true parameters of the system [xi (-), wn (rad/sec)] 

### Lay Out Problem Dimensionality ###
nInf = 4                     # Number of inferred variables [-]
nState = states.shape[0]     # Number of states [-]
nPar = nInf - nState         # Number of parameters [-]
samps = len(time)            # Number of system measurements [-]

## Define Transition and Emission Functions

In [3]:
def fx(x, dt, exc=None):
    """
    State transition model for a SDOF oscillator with a Bouc-Wen switch 
    state, given that alpha = 0, and therefore the Bouc-Wen component is 
    switched off.
      
    x = 1x4 vector of states (disp [m], vel [m/sec]) and parameters 
                to be inferred (log(xi),log(wn)). 
    dt = sampling rate [sec]
    exc = input excitation at current time step [m/sec^2]
    """
    if exc is None:
        exc = np.zeros(x[1].shape)
      
    par = np.exp(x[2:])         
    x1dot = x[0] + dt*x[1]
    x2dot = x[1] + dt*(-exc - (2*par[0]*par[1])*x[1] - np.square(par[1])*x[0])

    return np.concatenate((np.stack((x1dot, x2dot), axis=0)
                           , x[2:]), axis=0)

def hx(x):
    """
    Measurement model for a SDOF oscillator with a Bouc-Wen switch 
    state, given that alpha = 0, and therefore the Bouc-Wen component is 
    switched off.
      
    x = 1x4 vector of states (disp [m], vel [m/sec]) and parameters 
                to be inferred (log(xi),log(wn)). 
    """  
    par = np.exp(x[2:])
    resp = -(2*par[0]*par[1])*x[1] - np.square(par[1])*x[0]
    return np.eye(1)*resp # Dimensionality matching for FilterPy

In [4]:
def my_measUT(sigmas, Wm, Wc, noise_cov=None,
                        mean_fn=None, residual_fn=None):
    """
    This function is an optional input to the "update" step of the UKF
    algorithm in FilterPy, which allows for a user-generated calculation
    of the unscented transform, given a set of sigma points and their 
    weights. I define this function to enable the computation of the 
    unscented transform in the case where there is only one vector of 
    system observations. 
    
    This function works in conjunction with the UnscentedKalmanFilter 
    class, and follows the terminology therein.
    
    sigmas = Nx(2N+1) matrix containing the sigma points to be evaluated
    Wm = vector of weights on the sigma points for the computation  
         of the mean
    Wc = vector of weights on the sigma points for the computation
         of the covariance
    noise_cov = noise matrix added to the final computational covariance
                matrix
    mean_fn = Function that computes the mean of the provided sigma points
              and weights. Useful for state variables containin nonlinear 
              values which cannot be summed.
    residual_fn = Function that computes the residual (difference) between 
                  x and y. Useful for state variable that do not support 
                  subtraction. 
    """
    sigmas = sigmas[:,:,0] # Dimensionality matching for FilterPy
    kmax, n = sigmas.shape # Determine number of states (kmax) and
                           # number of sigma points (n)

    ## Approximate the Gaussian Mean ##
    try:
        if mean_fn is None:
            x = np.dot(Wm, sigmas)    # dot = \Sum^n_1 (W[k]*Xi[k])
        else:
            x = mean_fn(sigmas, Wm)
    except:
        print(sigmas)
        raise

    ## Approximate the Gaussian Covariance ##
    # The approximate covariance is the sum of the 
    # outer product of the residuals times the weights
    if residual_fn is np.subtract or residual_fn is None:
        # Fast Way
        y = sigmas - x[np.newaxis, :]
        P = np.dot(y.T, np.dot(np.diag(Wc), y))
    else:
        # Slow Way
        P = np.zeros((n, n))
        for k in range(kmax):
            y = residual_fn(sigmas[k], x)
            P += Wc[k] * np.outer(y, y)

    if noise_cov is not None:
        P += noise_cov

    return (x, P)

## Run UKF

In [5]:
### Load Prior Distributions on the Parameters ###
parPriors = np.loadtxt('../04-Data/parameter_priors.txt')

### Generate Storage Over Inferred States/Parameters ###
muHist = np.zeros((parPriors.shape[0],nInf, samps))
    # mean of the inferred parameters for each inference trial
    # over the observation period. This is what the EKF directly
    # outputs
stdHist = np.zeros((parPriors.shape[0],nInf, samps))
    # standard deviation of the inferred parameters for each 
    # inference trial over the observation period. This is 
    # what the EKF directly outputs
meanHist = np.zeros((parPriors.shape[0],nInf, samps))
    # mean of the inferred states and the underlying parameters
    # for each inference trial over the observation period. This
    # measure transforms the parameters to a lognormal distribution
    # such that the statistics on the true parameter values can be
    # extracted. 
modeHist = np.zeros((parPriors.shape[0],nInf, samps))
    # mode of the inferred states and the underlying parameters
    # for each inference trial over the observation period. This
    # measure transforms the parameters to a lognormal distribution
    # such that the statistics on the true parameter values can be
    # extracted. 
modStates = np.zeros((parPriors.shape[0],nInf, samps))
    # Response history of the inferred system given the input
    # excitation. Essentially, we're remodeling the behavior of 
    # the system given our selections on point estimates of the 
    # parameters from the posterior. 
runTime = np.zeros((parPriors.shape[0]))
    # Computational time for each inference trial. 

## For Each Inference Trial... ##
for j in range(parPriors.shape[0]):
    ### Create an Instance of the UKF Class ##
    sigPoints = MerweScaledSigmaPoints(nInf, alpha=0.001, beta=2, kappa=0)
        # Initialize sigma point generator class
    kf = UnscentedKalmanFilter(dim_x=nInf, dim_z=1, dt=dt, 
                               fx=fx, hx=hx,points=sigPoints)
        # Initialize UKF solver class with 8 states (2 dynamic, 2 parameters)
    
    ### Initialize UKF Priors ###
    mu0 = np.concatenate((np.zeros((nState,)), parPriors[j,:4:2]))
    P0 = np.diag(np.square(np.concatenate((np.array([0.05, 0.05])
                                 ,parPriors[j,1:5:2]))))
  
    kf.x = mu0    # Prior mean on the states and parameters
    kf.P = P0     # Prior covariacne on the states and parameters
    kf.Q = np.diag(np.concatenate((np.square(Q), 10**(-15)*np.ones(nPar))))
        # Adding a little jitter on the parameter transition density is
        # standard practice in the joint state-parameter estimation problem. 
        # Doing so avoids singularity in the covariance on the transition dynamics
    kf.R = R**2

    ## Store Initial Values ##
    muHist[j,:,0] = kf.x 
    stdHist[j,:,0] = np.sqrt(np.diag(kf.P))
    meanHist[j,:,0] = np.concatenate((kf.x[0:nState], np.exp(kf.x[nState:] 
                                        + np.square(stdHist[j,nState:,0])/2)))
    modeHist[j,:,0] = np.concatenate((kf.x[0:nState], np.exp(kf.x[nState:] 
                                        - np.square(stdHist[j,nState:,0]))))

    ### Run UKF Over Data ###
    t0 = timeit.time()
    try: 
        for i in range(1,samps):
            ## Predictor ##
            kf.predict(exc = inpAcc[i-1])
            ## Corrector ##
            kf.update(respAcc[i], UT=my_measUT)

            ## Store Results ##
            muHist[j,:,i] = kf.x
            stdHist[j,:,i] = np.sqrt(np.diag(kf.P))
            meanHist[j,:,i] = np.concatenate((kf.x[0:nState], 
                                    np.exp(kf.x[nState:] + np.square(stdHist[j,nState:,i])/2)))
            modeHist[j,:,i] = np.concatenate((kf.x[0:nState], np.exp(kf.x[nState:] 
                                    - np.square(stdHist[j,nState:,i]))))
    except:
        print('\nERROR: Numerical Instability')
        ## Store Results ##
        for i in range(1,samps):
            muHist[j,:,i] = np.array([0,0,-0.1054, 2.3])
            stdHist[j,:,i] = 0.01*np.ones(4)
            meanHist[j,:,i] = np.concatenate((muHist[j,:nState,i], 
                                    np.exp(muHist[j,nState:,i] + np.square(stdHist[j,nState:,i])/2)))
            modeHist[j,:,i] = np.concatenate((muHist[j,:nState,i], np.exp(muHist[j,nState:,i] 
                                    - np.square(stdHist[j,nState:,i]))))
        
    ### Rerun Model with Point Estimates (Mode) of Inferred Parameters ###
    modStates[j,:,0] = np.concatenate((np.zeros((2,)), np.log(modeHist[j,2:,-1])))
    for i in range(1,samps):
        modStates[j,:,i] = fx(modStates[j,:,i-1], dt, exc=inpAcc[i-1])   
    
    tf = timeit.time()
    runTime[j] = tf-t0

    ## Print Results Summary ##
    print('\nIteration %d' %(j))
    print('Computation Time = %.2f seconds' %((tf-t0)))
    print('Mode of Final Parameter Distributions = \n\txi = %.4f\n\twn = %.4f'
          %(modeHist[j,2,-1],modeHist[j,3,-1]))

np.savez(outFile, muHist = muHist,stdHist=stdHist, meanHist=meanHist, 
         modeHist=modeHist, modStates=modStates, runTime = runTime)


Iteration 0
Computation Time = 1.26 seconds
Mode of Final Parameter Distributions = 
	xi = 0.0897
	wn = 3.0069

Iteration 1
Computation Time = 1.27 seconds
Mode of Final Parameter Distributions = 
	xi = 0.0846
	wn = 3.0021

Iteration 2
Computation Time = 1.30 seconds
Mode of Final Parameter Distributions = 
	xi = 0.0739
	wn = 2.9952

Iteration 3
Computation Time = 1.23 seconds
Mode of Final Parameter Distributions = 
	xi = 0.0510
	wn = 2.9926

Iteration 4
Computation Time = 1.25 seconds
Mode of Final Parameter Distributions = 
	xi = 0.0486
	wn = 2.9922

Iteration 5
Computation Time = 1.27 seconds
Mode of Final Parameter Distributions = 
	xi = 0.0881
	wn = 3.0055

Iteration 6
Computation Time = 1.26 seconds
Mode of Final Parameter Distributions = 
	xi = 0.0508
	wn = 2.9926

Iteration 7
Computation Time = 1.21 seconds
Mode of Final Parameter Distributions = 
	xi = 0.0722
	wn = 2.9956

Iteration 8
Computation Time = 1.25 seconds
Mode of Final Parameter Distributions = 
	xi = 0.0832
	wn =

## Predictive Capacity of the Inferred Models
The goal of this section is to develop a prediction of the response behavior of the system to a secondary event, given the models which have been inferred from the primary excitation. 

### Load Inference Data
This becomes an optional start point in the code. If the data for the UKF has already been generated, it can simply be loaded in for the predictive analysis instead of rerunning the previous block of code. 

In [ ]:
outData = np.load(outFile + '.npz')

muHist = outData['muHist']         # inference history of untransformed state/par means
stdHist = outData['stdHist']       # inference history of state/par standard deviations
meanHist = outData['meanHist']     # inference history of transformed state/par means
modeHist = outData['modeHist']     # inference history of transformed state/par modes
modStates = outData['modStates']   # states that have been remodeled based on the final modes of the parameters

### Load Secondary Input Excitation

In [8]:
predInFile = '../04-Data/Linear/predInp_BLWN'
predOutFile = '../04-Data/Linear/predOutUKF'

infData = np.load(predInFile + '.npz')

dt = infData['dt']                            # time step [sec]
time = infData['time']                        # time history [sec]
predBase = infData['predInp']                 # observations on input acceleration [m/sec^2]
predStatesTrue = infData['predStatesPNoise']  # states (for post-prediction validation) [m,m/sec]
predRespTrue = infData['predAccPMNoise']      # observations on response acceleration [m/sec^2]
Q = infData['Qfactor']                        # process noise contributions, independent std. dev. per state [m,m/sec]
R = infData['Rfactor']                        # measurement noise contribution [m/sec^2]
m = infData['m']                              # mass [kg]
ics = infData['ics']                          # true initial conditions of the system [m, m/sec]
par = infData['par']                          # true parameters of the system [xi (-), wn (rad/sec)] 

### Generate Predictive Distribution on the States over Secondary Input

In [9]:
### Set Constants for Predictive Sampling ###
nPriors = muHist.shape[0]    # Number of inference trials [-]
nSamps = 500                 # Number of samples on the inference posterior [-]
seeds = [8192,3245]          # seeds for random number generator

### Generate Storage Over Predicted States ###
totalSamps = np.zeros((nSamps*nPriors, nState, len(time)))
    # Predicted states based on simulations results for all posterior
    # samples from all inference trials
meanPred = np.zeros((nPriors, nState, len(time)))
    # Mean of the predicted states for each inference trial.
stdPred = np.zeros((nPriors,nState, len(time)))
    # Standard deviation of the predicted states for each inference trial. 

### Run Predictive Trials on All Candidate Models ###
print('Predictive Distribution from Inferred Results')
for j in range(nPriors):
    print('Case %d'%(j))
    ## Random Samples on the States and Parameters, based on Inferred Posterior ##
    np.random.seed(seeds[0]+j)
    rSamp = np.random.multivariate_normal(np.zeros(nInf), np.eye(nInf), nSamps)
    predSamps = muHist[j,:,-1] + stdHist[j,:,-1]*rSamp

    ## Random Samples on the Transition Noise ##
    np.random.seed(seeds[1]+j)
    noise = Q.reshape(-1,1)*np.random.multivariate_normal(np.zeros(nState), np.eye(nState), 
                                                          (nSamps, len(time))).transpose((0, 2, 1))

    ## Prepare Response Storage ##
    predStates = np.zeros((nSamps, nState,len(time)))
    predStates[:,:,0] = predSamps[:,:nState]

    for i in range(nSamps):
        for tt in range(1,len(time)):
            predStates[i,:,tt] = fx(np.concatenate((predStates[i,:, tt-1], predSamps[i,nState:])), 
                                                 dt, exc = predBase[tt-1])[:nState] + noise[i,:,tt-1]
    
    ## Store Results from Predictive Sample Runs ##
    meanPred[j,:,:] = np.mean(predStates, axis = 0)
    stdPred[j,:,:] = np.sqrt(np.mean(np.square(predStates), axis=0) - np.square(meanPred[j,:,:])) 
    totalSamps[j*nSamps:(j+1)*nSamps,:,:] = predStates
    
### Remove Unstable Results from the Overall Assessment ###
# Candidate models can become unstable during inference (due to 
# computational issues such as singularities in the covariance 
# matrices) or manifest instability during predictive modeling
# due to combinations of the selected parameters which result in
# model divergence. Here we extract these cases so that they don't 
# interfere with the statistics of the main results. 
stabilityInd = np.ones(nPriors)
totalStabilityInd = np.ones(nPriors*nSamps)

print('\nIndices of Unstable Predictive Distributions:')
for i in range(nPriors):
    if (np.isnan(meanPred[i,0,-1])) or (np.absolute(meanPred[i,0,-1])>100) or (muHist[i,0,-1] == 0):
        stabilityInd[i] = 0 
        totalStabilityInd[i*nSamps:(i+1)*nSamps] = np.zeros(nSamps)
        print(i)

stableMeans = meanPred[stabilityInd != 0,:,:]
stableStds = stdPred[stabilityInd != 0,:,:]
stableSamps = totalSamps[totalStabilityInd != 0,:,:]

### Statistics on all Stable Cases ###
meanAll = np.mean(stableSamps, axis = 0)
stdAll = np.sqrt(np.mean(np.square(stableSamps), axis=0) - np.square(meanAll)) 

### Save Output ###
np.savez(predOutFile, meanPred = meanPred,stdPred=stdPred, 
         stableMeans=stableMeans, stableStds=stableStds, meanAll=meanAll, stdAll=stdAll)

Predictive Distribution from Inferred Results
Case 0
Case 1
Case 2
Case 3
Case 4
Case 5
Case 6
Case 7
Case 8
Case 9
Case 10
Case 11
Case 12
Case 13
Case 14
Case 15
Case 16
Case 17
Case 18
Case 19
Case 20
Case 21
Case 22
Case 23
Case 24
Case 25
Case 26
Case 27
Case 28
Case 29
Case 30
Case 31
Case 32
Case 33
Case 34
Case 35
Case 36
Case 37
Case 38
Case 39
Case 40
Case 41
Case 42
Case 43
Case 44
Case 45
Case 46
Case 47
Case 48
Case 49

Indices of Unstable Predictive Distributions:
